# Forecast Demo/PoC

Notebook for training different forecast models (Holt-Winters and ARIMAX) and parameters on insurance claims data and evaluating the resulting predictions on a hold out test set.

    Prepare the Data
    0. Import Packages
    1. Read Data
    2. Data Preparation
        2.1 Encoding
    3. Pre-Processing
        3.1 Correlation
        3.2 Train/Test Partitioning
    
    Traditional Time Series Models
    4. Train Holt-Winters Models
    5. ARIMAX Models
        5.1 Differencing
        5.2 Auto-Regression
        5.3 Moving Average
        5.4 Train ARIMA Models
    6. Holt-Winters & ARIMAX Comparison
    
    Automated Time Series Models
    7. Auto-ARIMAX
    8. Auto-SARIMAX



## 0. Import Packages

In [ ]:
##### 0.  Packages
import os
import pandas as pd
import numpy as np
from datetime import timedelta
from matplotlib import pyplot as plt

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import ExponentialSmoothing as HWES
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

import pmdarima as pm


In [ ]:
#####  1.  Read Data
os.chdir("C:\\Users\\jodickson\\OneDrive - Deloitte (O365D)\\Documents\\HBF\\Forecasting_PoC")
df0 = pd.read_csv('data\\insurance_claims_daily.csv', header=0, index_col=0)
df=df0

pd.set_option('display.max_columns', None)
#pd.reset_option('max_columns')
df.head()

## 2. Data Preparation

In [ ]:
#####  2. Data Preparation

# Target Feature:  
#   Daily Total Claim Amount 


#####  2.1 Encoding

# Encode dates
df.incident_date = pd.to_datetime(df.incident_date, format='%Y-%m-%d')


## 3. Pre-Processing

In [ ]:
######  3 Pre-Processing

## 3.1 Check for correlation amongst predictors

# Policy state - Weak correlation with OH pct
df[['total_claim_amount', 'policy_state_IL_pct', 'policy_state_IN_pct', 'policy_state_OH_pct']].corr()

# Sex - Weak correlation
df[['total_claim_amount', 'insured_male_pct', 'insured_female_pct']].corr()

# Eductaion - Weak correlation
df[['total_claim_amount','insured_High_School_pct', 'insured_JD_pct','insured_Associate_pct', 
          'insured_MD_pct', 'insured_Masters_pct', 'insured_PhD_pct','insured_College_pct']].corr()

# Age and Customer Months - some correlation with max
df[['total_claim_amount', 'age_mean', 'age_min', 'age_max', 
          'customer_months_mean', 'customer_months_min', 'customer_months_max']].corr()

# Age Mean and Max are correlated with Customer months mean and max 
# (the older a customer is, the longer they could have been a customer)
# Some correlations between  max age/months and claim amount

In [ ]:
##  3.2 Train and test split
# Create training set for modelling and hold-out test set for evaluation

train_pct = 0.8  
train = df.iloc[0:round(len(df)*train_pct)]
test = df.iloc[round(len(df)*train_pct):]


## 4. Train Holt-Winters Model

In [ ]:
#####  4.   Train Holt-Winters Model

hwdef = HWES(endog=train['total_claim_amount'], seasonal_periods=7, trend='add', seasonal='add', initialization_method="estimated")
hwmod = hwdef.fit()
hwfitted = hwmod.fittedvalues
train = pd.concat([train, pd.DataFrame({'HW_Fitted' : hwfitted})], axis=1)

# Out of Sample Test
hwforecast = hwmod.forecast(steps=len(test))
test = pd.concat([test, pd.DataFrame({'HW_Forecast' : hwforecast})], axis=1)


# Forecast Error
hw_rmse = np.sqrt(np.mean((test.total_claim_amount - test.HW_Forecast)**2))
hw_mae = np.mean(np.abs(test.total_claim_amount - test.HW_Forecast))
print("RMSE: " + str(round(hw_rmse)))
print("MAE: " + str(round(hw_mae)))

In [ ]:
# Chart whole data set with test set forecast
fig, ax = plt.subplots(figsize=(9,2.5))
npre = 4
ax.set(title='Total Claim Amount', xlabel='Date', ylabel='Million Dollars')
# Plot data points
plt.plot(df['incident_date'], df['total_claim_amount'], label='Observed',  markersize=1)
train.plot(x='incident_date', y='HW_Fitted', ax=ax, style='orange', label='Fitted Values', linewidth=1, linestyle='--')
# Plot predictions
test.plot(x='incident_date', y='HW_Forecast', ax=ax, style='g', label='Out of Sample HW Forecast')
legend = ax.legend(loc='lower right', fontsize='xx-small')


# # Chart test set with forecast
fig, ax = plt.subplots(figsize=(9,2.5))
npre = 4
ax.set(title='Total Claim Amount', xlabel='Date', ylabel='Million Dollars')
# Plot data points
plt.plot(test['incident_date'], test['total_claim_amount'], label='Observed', marker='o', markersize=4)
# Plot predictions
test.plot(x='incident_date', y='HW_Forecast', ax=ax, style='g', label='Out of Sample HW Forecast')
legend = ax.legend(loc='lower right', fontsize='xx-small')
plt.show()

## 5. ARIMAX Models

In [ ]:
#####  5.  ARIMAX

### 5.1  Check differencing the data th guide choice of I parameter

# Use AD Fuller test to test if time series is stationary
# Null hypothesis of the ADF test is that the time series is non-stationary

result = adfuller(train.total_claim_amount)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
# p < 0.5, so we can infer time series is stationary 

In [ ]:
# I - Integrated
# Plot total claim amount for 1st order differencing

# Original Series
plt.rcParams.update({'figure.figsize':(9,6), 'figure.dpi':120})
fig, axes = plt.subplots(2, 2, sharex=False)
axes[0, 0].plot(train.total_claim_amount.dropna()); axes[0, 0].set_title('Original Series')
plot_acf(train.total_claim_amount.dropna(), ax=axes[0, 1], lags=30)

# 1st Differencing
axes[1, 0].plot(train.total_claim_amount.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(train.total_claim_amount.diff().dropna(), ax=axes[1, 1], lags=30)
plt.show()

# 1st Order Differencing ACF Plot indicates over-differenced data (strong negative first lag)

In [ ]:
###  5.2 Auto-Regression

# PACF plot of 1st differenced series
plt.rcParams.update({'figure.figsize':(9,2.5), 'figure.dpi':120})

fig, axes = plt.subplots(1, 2, sharex=False)
axes[0].plot(train.total_claim_amount); axes[0].set_title('Original Series')
plot_pacf(train.total_claim_amount.dropna(), ax=axes[1], lags=20)
plt.show()

# Partial-autocorrelation, with sig. correlation at lag 1, so an AR terms is required

In [ ]:
###  5.3 Moving Average
plt.rcParams.update({'figure.figsize':(9,2.5), 'figure.dpi':120})
fig, axes = plt.subplots(1, 2, sharex=False)
axes[0].plot(train.total_claim_amount); axes[0].set_title('Original Data')
plot_acf(train.total_claim_amount.dropna(), ax=axes[1])
plt.show()

# Autocorrelation at lag 1 and (somewhat) at lag 2, so we need an MA term (use 1 MA term to be conservative)

In [ ]:
# 5.4 Train ARIMA X Models
# ARIMA (p,d,q) - (AR specification, Integration order, MA specification)

# Predictors (endogenous and exogenous)
endog = train['total_claim_amount']
exog_features = ['age_max', 'DOW__1', 'DOW__2', 'DOW__3', 'DOW__4', 'DOW__5', 'DOW__6']
exog = train[exog_features]

# Define and fit model
def1 = sm.tsa.statespace.SARIMAX(endog, exog, order=(1,0,1))
mod1 = def1.fit(disp=False)
print(mod1.summary())

In [ ]:
# Refit model after dropping the MA term 
def2 = sm.tsa.statespace.SARIMAX(endog, exog, order=(1,0,0))
mod2 = def2.fit(disp=False)

print(mod2.summary())
res2 = mod2.resid

In [ ]:
# Plot residual errors
fig, ax = plt.subplots(1,2, figsize=(9,2)) 
res2.plot(title="Residuals", ax=ax[0])
res2.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

# Actual vs Fitted
# Add fitted values onto training set
train = pd.concat([train, pd.DataFrame({'ARIMA_Fitted' : mod2.fittedvalues})], axis=1, ignore_index=False)

fig, ax = plt.subplots(figsize=(9,2.5))
plt.plot(train['incident_date'], train['total_claim_amount'] ,color='cadetblue', label='Actuals')
plt.plot(train['incident_date'], train['ARIMA_Fitted'], color='orange', label='Fitted Value')
legend = ax.legend(loc='upper left', shadow=False, fontsize='x-small')
ax.set_title('Actuals vs Fitted')
plt.show()

In [ ]:
# Out of Sample Test
forecast = mod2.get_forecast(steps=len(test), exog=test[exog_features], alpha=0.05)
forecast_ci = forecast.conf_int()
forecast_ci.columns = ["Lower Bound","Upper Bound"]

# Add forecasts onto test set
test = pd.concat([test, pd.DataFrame({'Forecast' : forecast.predicted_mean}), forecast_ci], axis=1, ignore_index=False)

# Forecast Error
arima_rmse = np.sqrt(np.mean((test.total_claim_amount - test.Forecast)**2))
arima_mae = np.mean(np.abs(test.total_claim_amount - test.Forecast))

print("RMSE: " + str(round(arima_rmse)))
print("MAE: " + str(round(arima_mae)))


In [ ]:
# Chart whole data set with test set forecast
fig, ax = plt.subplots(figsize=(9,2.5))
npre = 4
ax.set(title='Total Claim Amount', xlabel='Date', ylabel='Million Dollars')
# Plot data points
plt.plot(df['incident_date'], df['total_claim_amount'], label='Observed',  markersize=1)
train.plot(x='incident_date', y='ARIMA_Fitted', ax=ax, style='orange', label='Fitted Values', linewidth=1, linestyle='--')
# Plot predictions
test.plot(x='incident_date', y='Forecast', ax=ax, style='g', label='Out of Sample Forecast')
#ax.fill_between(x=test['incident_date'], y1=test['Lower Bound'], y2=test['Upper Bound'], color='g', alpha=0.1, label="95% Confidence")
legend = ax.legend(loc='lower right', fontsize='xx-small')


# Test set with forecast
fig, ax = plt.subplots(figsize=(9,2.5))
npre = 4
ax.set(title='Total Claim Amount', xlabel='Date', ylabel='Million Dollars')
# Plot data points
plt.plot(test['incident_date'], test['total_claim_amount'], label='Observed', marker='o', markersize=3, linewidth=1)
#test.plot(x='incident_date', y='total_claim_amount', ax=ax, label='Observed', marker='o', markersize=4)
# Plot predictions
test.plot(x='incident_date', y='Forecast', ax=ax, style='g', label='Out of Sample Forecast')
# Plot Confidence Intervals
ax.fill_between(x=test['incident_date'], y1=test['Lower Bound'], y2=test['Upper Bound'], color='g', alpha=0.05, label="95% Confidence")
legend = ax.legend(loc='lower right', fontsize='xx-small')
plt.show()

##  6. Holt-Winters & ARIMAX Comparison

In [ ]:
# 6. Comparison

# Test set with forecast
fig, ax = plt.subplots(figsize=(9, 2.5))
npre = 4
ax.set(title='Total Claim Amount', xlabel='Date', ylabel='Million Dollars')
# Plot data points
plt.plot(test['incident_date'], test['total_claim_amount'], label='Observed', marker='o', markersize=3, linewidth=1)
# Plot predictions
test.plot(x='incident_date', y='HW_Forecast', ax=ax, style='orange', label='Holt-Winters Forecast')
test.plot(x='incident_date', y='Forecast', ax=ax, style='g', label='ARIMAX Forecast')
# Plot Confidence Intervals
legend = ax.legend(loc='lower right', fontsize='xx-small')
plt.show()


In [ ]:
# Forecast Error
HW_mae = np.mean(np.abs(test.total_claim_amount - test.HW_Forecast))
ARIMAX_mae = np.mean(np.abs(test.total_claim_amount - test.Forecast))

print("MAE")
print("Holt-Winters: " + str(f"{round(HW_mae):,d}"))
print("ARIMAX:       " + str(f"{round(ARIMAX_mae):,d}"))

## 7. Auto-ARIMAX Model

In [ ]:
################
# 7. Auto-ARIMAX

model_auto = pm.auto_arima(train['total_claim_amount'], train[exog_features], start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model_auto.summary())


In [ ]:
# Add forecasts onto test set
train_pct = 0.8  
train = df.iloc[0:round(len(df)*train_pct)]
test = df.iloc[round(len(df)*train_pct):]

forecast_AX = model_auto.predict(n_periods=len(test), return_conf_int=False, exogenous = test[exog_features])

test = pd.concat([test, pd.DataFrame({'Forecast_AutoARIMAX' : forecast_AX}, index = test.index)], axis=1, ignore_index=False)


## 8. Auto-SARIMAX (Seasonal ARIMAX)

In [ ]:
# 8. Auto-SARIMAX

model_autos2 = pm.auto_arima(train['total_claim_amount'], train[['trend', 'age_max']], start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=7,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=True,    # No Seasonality
                      start_P=0, 
                      D=1, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model_autos2.summary())


In [ ]:
# Add forecasts onto test set
forecast_ASX = model_autos2.predict(n_periods=len(test), exogenous = test[['trend', 'age_max']])

test=pd.concat([test, pd.DataFrame({'Forecast_AutoSARIMAX' : forecast_ASX}, index = test.index)], axis=1, ignore_index=False)
test.head()

In [ ]:
# Compare Auto-ARIMA with Auto-ARIMAX

# Test set with forecast
fig, ax = plt.subplots(figsize=(9, 3))
npre = 4
ax.set(title='Total Claim Amount', xlabel='Date', ylabel='Million Dollars')
# Plot data points
plt.plot(test['incident_date'], test['total_claim_amount'], label='Observed', marker='o', markersize=3, linewidth=1)
# Plot predictions
test.plot(x='incident_date', y='Forecast_AutoARIMAX', ax=ax, style='orange', label='Auto-ARMIAX')
test.plot(x='incident_date', y='Forecast_AutoSARIMAX', ax=ax, style='g', label='Auto-SARIMAX')
# Plot Confidence Intervals
legend = ax.legend(loc='lower right', fontsize='xx-small')
plt.show()


In [ ]:
# Forecast Error
auto_arimax_mae = np.mean(np.abs(test.total_claim_amount - test.Forecast_AutoARIMAX))
auto_sarimax_mae = np.mean(np.abs(test.total_claim_amount - test.Forecast_AutoSARIMAX))

print("MAE")
print("Auto ARIMAX:  " + str(f"{round(auto_arimax_mae):,d}"))
print("Auto SARIMAX: " + str(f"{round(auto_sarimax_mae):,d}"))